In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as req
apikey = open('apikey.txt', 'r').readline()

In [116]:
def csv_to_dict(file_path = 'station_name_lat_lng.csv'):
    df = pd.read_csv(file_path)
    return df.set_index('station_name')[['lat', 'lng']].apply(tuple, axis=1).to_dict()

def dict_to_csv(d = {}, file_path = 'station_name_lat_lng.csv'):
    df = pd.DataFrame(columns=['station_name','lat','lng'])
    for key, value in d.items():
        df.loc[len(df.index)] = [key, value[0], value[1]]
    df.to_csv(file_path, index=False)

In [69]:
def parse_geocode(address_lat_lng, res):
        body = res.json()
        results = body['results']
        location = results[0]['geometry']['location']
        address_lat_lng['formatted_address'] = results[0]['formatted_address']
        address_lat_lng['lat'] = float(location['lat'])
        address_lat_lng['lng'] = float(location['lng'])

def get_geocode_by_address(address, city = 'San Francisco', state = 'CA'):
    address_lat_lng = {
        'formatted_address': '',
        'lat': None,
        'lng': None
    }
    if len(address) > 0:
        url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address.replace(" ", "+")},+{city.replace(" ", "+")},+{state}&key={apikey}'
        res = req.get(url)
        parse_geocode(address_lat_lng, res)
    return address_lat_lng

def get_geocode_by_lat_lng(lat, lng):
    address_lat_lng = {
        'formatted_address': '',
        'lat': None,
        'lng': None
    }
    if (type(lat) == int or float) and (type(lng) == int or float):
        url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={apikey}'
        res = req.get(url)
        parse_geocode(address_lat_lng, res)
    return address_lat_lng

In [178]:
def updater(name, d):
    if not name in d:
        address_lat_lng = get_geocode_by_address(name)
        d[name] = (address_lat_lng['lat'], address_lat_lng['lng'])
    
def sanitize_tripdata_csv(file_path, dict_path):
    df = pd.read_csv(file_path, usecols=['started_at', 'ended_at', 'start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng'], nrows=10)
    df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S.%f').apply(lambda x: x.replace(microsecond=0))
    df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S.%f').apply(lambda x: x.replace(microsecond=0))
    df.dropna(inplace=True)
    df.sort_values(by = ['started_at', 'ended_at'], ignore_index=True, inplace=True)
    d = csv_to_dict(dict_path)
    for index, row in df.iterrows():
        start_station_name = row['start_station_name']
        updater(start_station_name, d)
        df.loc[index, 'start_lat'] = d[start_station_name][0]
        df.loc[index, 'start_lng'] = d[start_station_name][1]
        end_station_name = row['end_station_name']
        updater(end_station_name, d)
        df.loc[index, 'end_lat'] = d[end_station_name][0]
        df.loc[index, 'end_lng'] = d[end_station_name][1]
    df.drop(['start_station_name', 'end_station_name'], axis=1, inplace=True)
    dict_to_csv(d, dict_path)
    df.to_csv(f'{file_path}-sanitized.csv', index=False)
    
def sanitize_crimedata_csv(file_path, dict_path):
    df = pd.read_csv(file_path, usecols=['incident_date', 'incident_datetime', 'analysis_neighborhood', 'incident_category'], nrows=10)
    df.dropna(inplace=True)
    df.rename(columns={'incident_category': 'category'}, inplace=True)
    df['started_at'] = pd.to_datetime(df['incident_date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d') + ' ' + pd.to_datetime(df['incident_datetime'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M:%S')
    df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
    df.insert(0, 'start_lng', 0)
    df.insert(0, 'start_lat', 0)
    df.sort_values(by='started_at', ignore_index=True, inplace=True)
    d = csv_to_dict(dict_path)
    for index, row in df.iterrows():
        analysis_neighborhood = row['analysis_neighborhood']
        updater(analysis_neighborhood, d)
        df.loc[index, 'start_lat'] = d[analysis_neighborhood][0]
        df.loc[index, 'start_lng'] = d[analysis_neighborhood][1]
    df.drop(['incident_date', 'incident_datetime', 'analysis_neighborhood'], axis=1, inplace=True)
    df.insert(0, 'started_at', df.pop('started_at'))
    dict_to_csv(d, dict_path)
    df.to_csv(f'{file_path}-sanitized.csv', index=False)

In [179]:
file_path = 'tripdata/202406-baywheels-tripdata.csv'
dict_path = 'station_name_lat_lng.csv'
sanitize_tripdata_csv(file_path, dict_path)

In [177]:
file_path = 'crimedata/sf-crimedata-2024-06-01-2024-06-30.csv'
dict_path = 'station_name_lat_lng.csv'
sanitize_crimedata_csv(file_path, dict_path)